In [1]:
pip install tensorflow opencv-python matplotlib numpy flask


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Pc\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install split-folders


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Pc\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Split into Train & Validation

In [12]:
import os
import shutil
import splitfolders

# Path to your dataset
DATASET_PATH = r"C:\Users\Pc\OneDrive\Desktop\driver drowsness\Driver Drowsiness Dataset (DDD)"

# Output folder (it will create train/val folders)
OUTPUT_PATH = r"C:\Users\Pc\OneDrive\Desktop\driver drowsness\DDD_split"

# Split into 80% train and 20% val
splitfolders.ratio(DATASET_PATH, output=OUTPUT_PATH, seed=42, ratio=(.8, .2))


# Load Dataset in Keras

In [13]:
import tensorflow as tf

train_dir = r"C:\Users\Pc\OneDrive\Desktop\driver drowsness\DDD_split\train"
val_dir   = r"C:\Users\Pc\OneDrive\Desktop\driver drowsness\DDD_split\val"

# Load train & validation sets
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32
)
print(train_ds.class_names)



Found 33434 files belonging to 2 classes.
Found 8359 files belonging to 2 classes.
['Drowsy', 'Non Drowsy']


In [3]:
# Normalize (rescale pixel values)
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

#  Train Model

In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(base.input, output)

for layer in base.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=10)

model.save("drowsiness_model.h5")


Epoch 1/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 623s 593ms/step - accuracy: 0.9887 - loss: 0.0369 - val_accuracy: 0.9977 - val_loss: 0.0106
Epoch 2/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 581s 556ms/step - accuracy: 0.9989 - loss: 0.0043 - val_accuracy: 0.9994 - val_loss: 0.0032
Epoch 3/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 565s 540ms/step - accuracy: 0.9991 - loss: 0.0028 - val_accuracy: 0.9994 - val_loss: 0.0013
Epoch 4/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 574s 549ms/step - accuracy: 0.9988 - loss: 0.0040 - val_accuracy: 0.9947 - val_loss: 0.0125
Epoch 5/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 571s 546ms/step - accuracy: 0.9996 - loss: 0.0013 - val_accuracy: 0.9999 - val_loss: 7.3276e-04
Epoch 6/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 571s 546ms/step - accuracy: 1.0000 - loss: 5.4652e-05 - val_accuracy: 0.9996 - val_loss: 5.4548e-04
Epoch 7/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 573s 548ms/step - accuracy: 1.0000 - loss: 1.5567e-05 - val_accuracy: 0.9998 - val_loss: 4.9747e-04
Epoch 8/10
1045/1045 ━━━━━━━━━━━━━━━━━━━━ 

# Test on single images

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_path = r"C:\Users\Pc\OneDrive\Desktop\driver drowsness\DDD_split\train\Drowsy\ZC1352.png"   # replace with your own
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)

prediction = model.predict(x)[0][0]
print("Prediction:", "Non Drowsy" if prediction > 0.5 else "Drowsy")